# Running ETL to Build the Document Corpus

This notebook walks through the process for setting up the corpus of Full Stack documents that the bot searches over.

In each case, we have to
- Extract data from its natural habitat, like YouTube or GitHub
- Transform it into a format that is useful for our purposes
- Load it into our database in that format

hence the acronym "ETL".

In [1]:
!make secrets  # you'll need credentials for Mongo and Modal to run this
db, collection = "fsdl-dev", "ask-fsdl"  # we run this in a dev database

# drop the collection if it exists, it's just dev
!modal run app.py::drop_docs --db {db} --collection {collection}

###
# 🥞: Loaded config from .env.dev
###
python -m pip install -qqq -r requirements.txt
###
# 🥞: If you haven't gotten a Modal token yet, run make modal-token
###
Verifying token against https://api.modal.com
Token verified successfully
Token written to /Users/zanqiliang/.modal.toml
bash tasks/setup_environment_modal.sh 
###
# 🥞: Setting up modal environment dev
###
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /Users/zanqiliang/.pyenv/versions/ask-fsdl/lib/python3.10/site-packages/moda │
│ l/cli/environment.py:51 in create                                            │
│                                                                              │
│   50 def create(name: Annotated[str, typer.Argument(help="Name of the new en │
│ ❱ 51 │   environments.create_environment(name)                               │
│   52 │   typer.echo(f"Environment created: {name}")                          │
│                                                               

In [2]:
import json
from pathlib import Path
import pprint

from etl import markdown, pdfs, shared, videos
from etl.shared import display_modal_image

pp = pprint.PrettyPrinter(indent=2)

## PDFs: arXiV Papers

```bash
!modal run etl/pdfs.py --json-path data/llm-papers.json
```

In [3]:
display_modal_image(shared.image)

In [4]:
display_modal_image(pdfs.image)

In [5]:
papers_path = Path("data") / "llm-papers.json"

with open(papers_path) as f:
    pdf_infos = json.load(f)

pdf_infos[:100:20]

[{'tags': ['Multimodal', 'Vision', 'Internals'],
  'title': 'On the Hidden Mystery of OCR in Large Multimodal Models',
  'url': 'https://arxiv.org/abs/2305.07895'},
 {'tags': ['Evaluation'],
  'title': 'Asking Crowdworkers to Write Entailment Examples: The Best of Bad Options',
  'url': 'https://aclanthology.org/2020.aacl-main.68/'},
 {'tags': ['Prompting', 'Critical', 'Philosophy', 'Simulation'],
  'title': 'Inducing anxiety in large language models increases exploration and bias',
  'url': 'https://arxiv.org/abs/2304.11111v1'},
 {'tags': ['Ahead of Its Time'],
  'title': 'PROGRAMS WITH COMMON SENSE',
  'url': 'http://jmc.stanford.edu/articles/mcc59/mcc59.pdf'},
 {'tags': ['Ahead of Its Time', 'External Memory', 'Reasoning', 'Philosophy'],
  'title': 'Heuristic Problem Solving By Computer',
  'url': 'https://iiif.library.cmu.edu/file/Simon_box00065_fld04954_bdl0001_doc0001/Simon_box00065_fld04954_bdl0001_doc0001.pdf'}]

In [6]:
[p for p in pdf_infos if 'cmu' in p['url']]

[{'tags': ['Philosophy', 'Reasoning'],
  'title': 'Bounded Rationality',
  'url': 'https://iiif.library.cmu.edu/file/Simon_box00008_fld00585_bdl0001_doc0001/Simon_box00008_fld00585_bdl0001_doc0001.pdf'},
 {'tags': ['Ahead of Its Time', 'External Memory', 'Reasoning', 'Philosophy'],
  'title': 'Heuristic Problem Solving By Computer',
  'url': 'https://iiif.library.cmu.edu/file/Simon_box00065_fld04954_bdl0001_doc0001/Simon_box00065_fld04954_bdl0001_doc0001.pdf'}]

In [13]:
with pdfs.stub.run():
    # first, we enrich the paper data by finding direct PDF URLs where we can
    paper_data = pdfs.get_pdf_url.map(
        # pdf_infos[::25], #subsampling to run faster
        pdf_infos,
        return_exceptions=True,
    )
    extracts = pdfs.extract_pdf.map(paper_data, return_exceptions=True)
    # extracts = [sublist for sublist in extracts]
    # then we turn the PDFs into JSON documents
    documents = shared.unchunk(  # each pdf creates a list of documents, one per page, so we flatten
        # after we run the extract_pdf function on Modal to get those pages
        extracts
    )

Output()

✓ Initialized. View app at https://modal.com/apps/ap-Cu86mD58fPeDVpv4ux6xyl

Output()

✓ Created objects.
├── 🔨 Created extract_pdf.
├── 🔨 Created mount /Users/zanqiliang/ask-fsdl/docstore.py, /Users/zanqiliang/ask-fsdl/utils.py
├── 🔨 Created mount /Users/zanqiliang/ask-fsdl/etl
├── 🔨 Created fetch_papers.
└── 🔨 Created get_pdf_url.

Output()

...

requests.exceptions.SSLError: HTTPSConnectionPool(host='iiif.library.cmu.edu', port=443): Max retries exceeded with url: /file/Simon_box00065_fld04954_bdl0001_doc0001/Simon_box00065_fld04954_bdl0001_doc0001.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 467, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 1092, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.10/site-packages/urllib3/connection.py", line 642, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "/usr/local/lib/python3.10/site-packages/urllib3/connection.py", line 783, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
  File "/usr/local/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 469, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(sock, context, tls_in_tls, server_hostname)
  File "/usr/local/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 513, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "/usr/local/lib/python3.10/ssl.py", line 513, in wrap_socket
    return self.sslsocket_class._create(
  File "/usr/local/lib/python3.10/ssl.py", line 1071, in _create
    self.do_handshake()
  File "/usr/local/lib/python3.10/ssl.py", line 1342, in do_handshake
    self._sslobj.do_handshake()
ssl.SSLCertVerificationError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 491, in _make_request
    raise new_e
urllib3.exceptions.SSLError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)

The above exception was the direct cause of the following exception:



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/requests/adapters.py", line 486, in send
    resp = conn.urlopen(
  File "/usr/local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 844, in urlopen
    retries = retries.increment(
  File "/usr/local/lib/python3.10/site-packages/urllib3/util/retry.py", line 515, in increment
    raise MaxRetryError(_pool, url, reason) from reason  # type: ignore[arg-type]
urllib3.exceptions.MaxRetryError: HTTPSConnectionPool(host='iiif.library.cmu.edu', port=443): Max retries exceeded with url: /file/Simon_box00008_fld00585_bdl0001_doc0001/Simon_box00008_fld00585_bdl0001_doc0001.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):


  File "/pkg/modal/_container_entrypoint.py", line 351, in handle_input_exception
    yield
  File "/pkg/modal/_container_entrypoint.py", line 437, in run_inputs
    res = imp_fun.fun(*args, **kwargs)
  File "/root/etl/pdfs.py", line 82, in extract_pdf
    loader = PyPDFLoader(pdf_url)
  File "/usr/local/lib/python3.10/site-packages/langchain/document_loaders/pdf.py", line 150, in __init__
    super().__init__(file_path)
  File "/usr/local/lib/python3.10/site-packages/langchain/document_loaders/pdf.py", line 83, in __init__
    r = requests.get(self.file_path)
  File "/usr/local/lib/python3.10/site-packages/requests/api.py", line 73, in get
    return request("get", url, params=params, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/requests/api.py", line 59, in request
    return session.request(method=method, url=url, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/requests/sessions.py", line 589, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python3.10/site-packages/requests/sessions.py", line 703, in send
    r = adapter.send(request, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/requests/adapters.py", line 517, in send
    raise SSLError(e, request=request)
requests.exceptions.SSLError: HTTPSConnectionPool(host='iiif.library.cmu.edu', port=443): Max retries exceeded with url: /file/Simon_box00008_fld00585_bdl0001_doc0001/Simon_box00008_fld00585_bdl0001_doc0001.pdf (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))


✓ App completed.

In [15]:
pp.pprint(documents[0]["metadata"])

{ 'arxiv_id': '2305.07895',
  'date': datetime.datetime(2023, 6, 19, 3, 36, 8, tzinfo=datetime.timezone.utc),
  'full-title': 'On the Hidden Mystery of OCR in Large Multimodal Models - p0',
  'ignore': False,
  'is_endmatter': False,
  'page': 0,
  'sha256': '0f8257ead372b2b93b7b1a257e70f6ed36f37a231b45a17852bb50672cc5bfef',
  'source': 'https://arxiv.org/abs/2305.07895',
  'title': 'On the Hidden Mystery of OCR in Large Multimodal Models'}


In [20]:
from IPython.display import IFrame

IFrame(src=documents[0]["metadata"]["source"], width=800, height=400)

In [ ]:
with shared.stub.run():
    # we split our document list into 10 pieces, so that we don't open too many connections
    chunked_documents = shared.chunk_into(documents, 10)
    list(shared.add_to_document_db.map(chunked_documents, kwargs={"db": db, "collection": collection}))

In [ ]:
with shared.stub.run():
  import docstore
   # pull only arxiv papers
  query = { "metadata.source": { "$regex": "arxiv\.org", "$options": "i" } }
  # project out the text field, it can get large
  projection = {"text": 0}
  # get just one result to show it worked
  result = docstore.query_one(query, projection, db=db, collection=collection)

pp.pprint(result)

## Markdown Files: Lectures

```bash
!modal run etl/markdown.py --json-path data/lectures-2022.json
```

In [ ]:
display_modal_image(markdown.image)

In [ ]:
markdown_path = Path("data") / "lectures-2022.json"

with open(markdown_path) as f:
  markdown_corpus = json.load(f)

website_url, md_url = (
  markdown_corpus["website_url_base"],
  markdown_corpus["md_url_base"],
)

lectures = markdown_corpus["lectures"]

lectures[0]

In [ ]:
with markdown.stub.run():
    documents = (
        shared.unchunk(  # each lecture creates multiple documents, one per section so we flatten
            markdown.to_documents.map(
                lectures,
                kwargs={"website_url": website_url, "md_url": md_url},
                return_exceptions=True,
            )
        )
    )

In [ ]:
pp.pprint(documents[1]["metadata"])

In [ ]:
from IPython.display import IFrame

IFrame(src=documents[1]["metadata"]["source"], width=800, height=400)

In [ ]:
with shared.stub.run():
    chunked_documents = shared.chunk_into(documents, 10)
    list(shared.add_to_document_db.map(chunked_documents, kwargs={"db": db, "collection": collection}))

In [ ]:
with shared.stub.run():
  import docstore
  # pull only lectures
  query = { "metadata.source": { "$regex": "lecture", "$options": "i" } }
  # project out the text field, it can get large
  projection = {"text": 0}
  # get just one result to show it worked
  result = docstore.query_one(query, projection, db=db, collection=collection)

pp.pprint(result)

## Videos: YouTube Transcripts

In [ ]:
display_modal_image(videos.image)

In [ ]:
videos_path = Path("data") / "videos.json"

with open(videos_path) as f:
    video_infos = json.load(f)

video_ids = [video["id"] for video in video_infos]

video_infos[0]

In [ ]:
with videos.stub.run():
    documents = (
        shared.unchunk(  # each lecture creates multiple documents, one per chapter, so we flatten
            videos.extract_subtitles.map(
                video_infos[-3:],  # subsampling to run faster
                return_exceptions=True,
            )
        )
    )

In [ ]:
pp.pprint(documents[1]["metadata"])

In [ ]:
from IPython.display import YouTubeVideo

id_str, time_str = documents[1]["metadata"]["source"].split("?v=")[-1].split("&t=")
YouTubeVideo(id_str, start=int(time_str.strip("s")), width=800, height=400)

In [ ]:
with shared.stub.run():
    chunked_documents = shared.chunk_into(documents, 10)
    list(shared.add_to_document_db.map(chunked_documents, kwargs={"db": db, "collection": collection}))

In [ ]:
with shared.stub.run():
  import docstore
  # pull only lectures
  query = { "metadata.source": { "$regex": "youtube", "$options": "i" } }
  # project out the text field, it can get large
  projection = {"text": 0}
  # get just one result to show it worked
  result = docstore.query_one(query, projection, db=db, collection=collection)

pp.pprint(result)